# Phase : Analyse des données

In [5]:
!pip install soumabkar-films-sdk==0.0.3


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from soumabkar_films_sdk import MovieClient, MovieConfig

# Configuration avec l’URL de votre API (Render ou locale)
config = MovieConfig(movie_base_url="https://movie-backend-cm2v.onrender.com")
client = MovieClient(config=config)

d:\training\Udemy\De la Modélisation SQLAlchemy a l API et a la Data App  maitrisez Python grace un projet 100pour100 pratique, réel et complet\projet_code\films-analytics\.venv\Lib\site-packages\pydantic\_internal\_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


MOVIE_API_BASE_URL in MovieConfig init: https://movie-backend-cm2v.onrender.com


In [ ]:
client.health_check()
# Retourne : {"status": "ok"}

In [13]:
# Récupération d’un film pour test
movie = client.get_movie(1)
print(f"Titre : {movie.title}")
print(f"Genres : {movie.genres}")

Titre : Toy Story (1995)
Genres : Adventure|Animation|Children|Comedy|Fantasy


In [14]:
# On récupère une grande portion de la table (à ajuster selon la taille réelle)
ratings_df = client.list_ratings(limit=100, output_format="pandas")
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [15]:
import time
import pandas as pd

In [16]:
# 1. Récupérer le total d’évaluations
total_ratings = client.get_analytics().rating_count
batch_size = 1000
all_ratings = []

# 2. Boucle sur les batches avec pause
for skip in range(0, total_ratings, batch_size):
    print(f"Téléchargement des lignes {skip} à {skip + batch_size}...")
    batch_df = client.list_ratings(skip=skip, limit=batch_size, output_format="pandas")
    all_ratings.append(batch_df)
    time.sleep(0.5)  # pause de 0.5 seconde

# 3. Concaténer tous les résultats
complete_ratings_df = pd.concat(all_ratings, ignore_index=True)

Téléchargement des lignes 0 à 1000...
Téléchargement des lignes 1000 à 2000...
Téléchargement des lignes 2000 à 3000...
Téléchargement des lignes 3000 à 4000...
Téléchargement des lignes 4000 à 5000...
Téléchargement des lignes 5000 à 6000...
Téléchargement des lignes 6000 à 7000...
Téléchargement des lignes 7000 à 8000...
Téléchargement des lignes 8000 à 9000...
Téléchargement des lignes 9000 à 10000...
Téléchargement des lignes 10000 à 11000...
Téléchargement des lignes 11000 à 12000...
Téléchargement des lignes 12000 à 13000...
Téléchargement des lignes 13000 à 14000...
Téléchargement des lignes 14000 à 15000...
Téléchargement des lignes 15000 à 16000...
Téléchargement des lignes 16000 à 17000...
Téléchargement des lignes 17000 à 18000...
Téléchargement des lignes 18000 à 19000...
Téléchargement des lignes 19000 à 20000...
Téléchargement des lignes 20000 à 21000...
Téléchargement des lignes 21000 à 22000...
Téléchargement des lignes 22000 à 23000...
Téléchargement des lignes 23000 à

In [17]:
# Agréger rating_count par userId, chunk par chunk

# 1. Récupérer le total d’évaluations
total_ratings = client.get_analytics().rating_count
batch_size = 1000

# 2. Dictionnaire pour accumuler les totaux par userId
from collections import defaultdict
user_rating_counts = defaultdict(int)

# 3. Parcours des chunks
for skip in range(0, total_ratings, batch_size):
    print(f"Traitement du batch {skip} à {skip + batch_size}...")
    batch_df = client.list_ratings(skip=skip, limit=batch_size, output_format="pandas")
    
    # Comptage des évaluations par utilisateur dans le batch
    batch_counts = batch_df['userId'].value_counts()
    
    # Mise à jour du compteur global
    for user_id, count in batch_counts.items():
        user_rating_counts[user_id] += count
    
    time.sleep(0.5)

# 4. Conversion en DataFrame finale
ratings_per_user = pd.DataFrame(list(user_rating_counts.items()), columns=["userId", "rating_count"])

ratings_per_user

Traitement du batch 0 à 1000...
Traitement du batch 1000 à 2000...
Traitement du batch 2000 à 3000...
Traitement du batch 3000 à 4000...
Traitement du batch 4000 à 5000...
Traitement du batch 5000 à 6000...
Traitement du batch 6000 à 7000...
Traitement du batch 7000 à 8000...
Traitement du batch 8000 à 9000...
Traitement du batch 9000 à 10000...
Traitement du batch 10000 à 11000...
Traitement du batch 11000 à 12000...
Traitement du batch 12000 à 13000...
Traitement du batch 13000 à 14000...
Traitement du batch 14000 à 15000...
Traitement du batch 15000 à 16000...
Traitement du batch 16000 à 17000...
Traitement du batch 17000 à 18000...
Traitement du batch 18000 à 19000...
Traitement du batch 19000 à 20000...
Traitement du batch 20000 à 21000...
Traitement du batch 21000 à 22000...
Traitement du batch 22000 à 23000...
Traitement du batch 23000 à 24000...
Traitement du batch 24000 à 25000...
Traitement du batch 25000 à 26000...
Traitement du batch 26000 à 27000...
Traitement du batch 270

,userId,rating_count
0,6,314
1,1,232
2,4,216
3,7,152
4,5,44
...,...,...
605,604,100
606,608,831
607,607,187
608,610,1302
